In [1]:
import random

random.seed(42)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import time

from PIL import Image
import imagesize

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras import layers, models

2024-06-05 20:57:38.172865: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
bloodcells = os.listdir("bloodcells_dataset")
bloodcells = [x for x in bloodcells if x not in ['.DS_Store', 'All_Images']]

bloodcells

['basophil',
 'neutrophil',
 'ig',
 'monocyte',
 'eosinophil',
 'erythroblast',
 'lymphocyte',
 'platelet']

In [3]:
df = pd.DataFrame(data = {'images': [np.nan], 'type': [np.nan]})

for i in range(len(bloodcells)):

    images = os.listdir('bloodcells_dataset/' + bloodcells[i]) # jpg string paths
    
    images_df = pd.DataFrame(data = {'images': images, 'type': bloodcells[i]})
    
    df = pd.concat([df, images_df])

df = df.dropna(how = 'all')

le = LabelEncoder()
df['type'] = le.fit_transform(df['type'])

df['width'] = df['images'].apply(lambda x: imagesize.get('bloodcells_dataset/All_Images/' + x)[0])
df['height'] = df['images'].apply(lambda x: imagesize.get('bloodcells_dataset/All_Images/' + x)[1])

df = df[df['images'].str.contains('copy') == False]
df = df.reset_index(drop = True)


df

,images,type,width,height
0,BA_689200.jpg,0,360,363
1,BA_883452.jpg,0,360,363
2,BA_382161.jpg,0,366,369
3,BA_175579.jpg,0,360,363
4,BA_775722.jpg,0,360,363
...,...,...,...,...
17087,PLATELET_495918.jpg,7,360,363
17088,PLATELET_897238.jpg,7,360,363
17089,PLATELET_750430.jpg,7,360,363
17090,PLATELET_810431.jpg,7,360,363


In [4]:
# Visuals for count of height/width and count of each blood cell type
# Maybe use function to plot

# df[['height', 'width']].value_counts().reset_index(name = 'count') 
# df[['type']].value_counts().reset_index(name = 'count')

In [5]:
class Sampling:

    def __init__(self, data, sampling_method):

        self.data = data
        
        self.sampling_method = sampling_method

    def sample_data(self, sampling_percent = 0.8):

        df = self.data.copy()

        category_counts = df[['type']].value_counts().reset_index(name = 'count')

        train = pd.DataFrame(np.nan, index = [0], columns = list(df.columns))

        if self.sampling_method == 'weighted':

            for i in range(len(category_counts)):

                type = category_counts['type'][i]
            
                if category_counts['count'][i] >= 2000:
                    add_samples = df[df['type'] == type].sample(1500)
                else: 
                    add_samples = df[df['type'] == type].sample(1000)
            
                train = pd.concat([train, add_samples])

        elif self.sampling_method == 'proportional': 

            num_samples = int(sampling_percent * len(df))
            
            category_counts['prop'] = category_counts['count'] / len(df)
            category_counts['prop_samples'] = category_counts['prop'] * num_samples
            category_counts['prop_samples'] = category_counts['prop_samples'].astype('int32')

            for i in range(len(category_counts)):

                type = category_counts['type'][i]
            
                samples = category_counts['prop_samples'][i]
            
                add_samples = df[df['type'] == type].sample(samples)
            
                train = pd.concat([train, add_samples])

        train = train.dropna(how = 'all')

        float_cols = train.select_dtypes(np.number)

        train[float_cols.columns] = float_cols.astype('int32')

        test = df[~df['images'].isin(train['images'])]    

        return train, test
            

In [6]:
weighted_sampling = Sampling(df, 'weighted')
proportional_sampling = Sampling(df, 'proportional')

weighted_train, weighted_test = weighted_sampling.sample_data()
prop_train, prop_test = proportional_sampling.sample_data()

In [7]:
class Convert_Images:

    def __init__(self, data):

        self.data = data

    def convert(self, resize):

        df = self.data.copy()

        resize_pixels = (resize, resize)

        images = np.array([np.array(Image.open('bloodcells_dataset/All_Images/' + image).resize(resize_pixels)) for image in df['images']])

        images = tf.convert_to_tensor(images / 255.0)

        labels = tf.convert_to_tensor(df['type'])

        return images, labels

In [8]:
start = time.time()

weighted_train_image_df, weighted_test_image_df = Convert_Images(weighted_train), Convert_Images(weighted_test)
prop_train_image_df, prop_test_image_df = Convert_Images(prop_train), Convert_Images(prop_test)


resize_pixels = 32

weighted_train_images, weighted_train_labels = weighted_train_image_df.convert(resize_pixels)
weighted_test_images, weighted_test_labels = weighted_test_image_df.convert(resize_pixels)
prop_train_images, prop_train_labels = prop_train_image_df.convert(resize_pixels)
prop_test_images, prop_test_labels = prop_test_image_df.convert(resize_pixels)

time.time() - start

2024-06-05 20:58:16.876651: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


93.01899695396423

In [9]:
# Opening an image
# test_img = Image.open('bloodcells_dataset/All_Images/' + df['images'][14000])

# Resize
# test_img.resize((320, 320))

# Crop
# test_img.crop((20, 20, 340, 340))

In [10]:
def train_model(model, train_data, train_labels, test_data, test_labels, optimizer = 'adam', epochs = 5, batch_size = 64):

    model.compile(optimizer = optimizer,
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                  metrics = ['accuracy'])

    history = model.fit(train_data, 
                        train_labels, 
                        epochs = epochs, 
                        batch_size = batch_size)

    predictions = (model.predict(test_data)).argmax(axis = 1)

    test_accuracy = np.sum(predictions == test_labels) / len(test_labels)

    return history, predictions, test_accuracy

In [11]:
# 1st of 3 models
# Simple model
# One for weighted sampling and proportional sampling

# Simple Model - Weighted Sampling
sm_w = models.Sequential(
    
    [
        layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (resize_pixels, resize_pixels, 3)),
        
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(32, (3, 3), activation = 'relu'),
        
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(64, (3, 3), activation = 'relu'),

        # flatten into 1d array
        layers.Flatten(),

        # Neural network
        layers.Dense(64, activation = 'relu'),

        layers.Dropout(rate = 0.2),
        
        # 8 different categories
        layers.Dense(8) 
    ]
    
)

# Simple Model - Proportional Sampling
sm_p = models.clone_model(sm_w)

In [12]:
sm_w_history, sm_w_predictions, sm_w_test_accuracy = train_model(sm_w,
                                                                 weighted_train_images,
                                                                 weighted_train_labels,
                                                                 weighted_test_images,
                                                                 weighted_test_labels,
                                                                 epochs = 20)

Epoch 1/20
157/157 [==============================] - 5s 29ms/step - loss: 1.4757 - accuracy: 0.4352
Epoch 2/20
157/157 [==============================] - 5s 30ms/step - loss: 0.9022 - accuracy: 0.6573
Epoch 3/20
157/157 [==============================] - 6s 39ms/step - loss: 0.7530 - accuracy: 0.7174
Epoch 4/20
157/157 [==============================] - 5s 31ms/step - loss: 0.6512 - accuracy: 0.7605
Epoch 5/20
157/157 [==============================] - 5s 32ms/step - loss: 0.5969 - accuracy: 0.7830
Epoch 6/20
157/157 [==============================] - 6s 35ms/step - loss: 0.5369 - accuracy: 0.8049
Epoch 7/20
157/157 [==============================] - 6s 37ms/step - loss: 0.4989 - accuracy: 0.8138
Epoch 8/20
157/157 [==============================] - 5s 31ms/step - loss: 0.4877 - accuracy: 0.8222
Epoch 9/20
157/157 [==============================] - 5s 30ms/step - loss: 0.4512 - accuracy: 0.8395
Epoch 10/20
157/157 [==============================] - 5s 30ms/step - loss: 0.4210 - accura

In [ ]:
sm_p_history, sm_p_predictions, sm_p_test_accuracy = train_model(sm_p,
                                                                 prop_train_images,
                                                                 prop_train_labels,
                                                                 prop_test_images,
                                                                 prop_test_labels, 
                                                                 epochs = 20)

Epoch 1/20
 72/214 [=========>....................] - ETA: 9s - loss: 1.7062 - accuracy: 0.3618

In [ ]:
sm_w_test_accuracy, sm_p_test_accuracy